## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [11]:
import numpy as np

A = np.random.randint(0, 1000, size=(1_000_000,))
A

array([149, 682, 763, ..., 254, 663, 806])

In [12]:
def f1(A):
    acc, cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt += 1
    return acc / cnt

In [13]:
%timeit f1(A)

1.46 s ± 82.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%lprun -f f1 f1(A)

In [15]:
def f2(A):
    return sum(A) / len(A) + 100

In [16]:
%timeit f2(A)

467 ms ± 32.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
def f3(A):
    return A.mean() + 100

In [20]:
%timeit f3(A)

1.81 ms ± 26.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
import numba

In [22]:
@numba.njit
def f4(A):
    acc, cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt += 1
    return acc / cnt

In [23]:
%timeit f4(A)

1.49 ms ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [30]:
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)

def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

g(df)

,col1,col2,col3,col4,key
1,37,679,612,2,a
2,469,261,361,237,a
20,503,682,175,114,a
25,596,515,375,838,a
29,377,516,43,537,a
...,...,...,...,...,...
1999959,230,63,277,158,e
1999975,536,516,207,811,e
1999982,388,872,969,175,e
1999983,465,428,136,429,e


In [28]:
%timeit g(df)

1.37 s ± 41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%lprun -f g g(df)

In [44]:
(df['key'] != 'f') & (df['key'] != 'g')

0           True
1           True
2           True
3           True
4           True
           ...  
1999995     True
1999996    False
1999997     True
1999998     True
1999999    False
Name: key, Length: 2000000, dtype: bool

In [45]:
def g1(df):
    return df[(df['key'] != 'f') & (df['key'] != 'g')]

g1(df)

,col1,col2,col3,col4,key
0,907,470,219,691,c
1,37,679,612,2,a
2,469,261,361,237,a
3,283,863,729,761,d
4,162,302,220,52,b
...,...,...,...,...,...
1999993,188,761,460,201,c
1999994,805,903,217,928,e
1999995,457,7,482,348,c
1999997,804,94,741,606,d


In [46]:
%timeit g1(df)

558 ms ± 21.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Лабораторная работа 3

In [1]:
!pip install line_profiler

In [9]:
%load_ext line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [10]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


#### [версия 2]
* Уточнены формулировки задач 1, 3, 4